# Basic Browser tool usage with Browser-Use SDK

## Overview

In this tutorial we will learn how to use the open source Browser-Use SDK with Amazon Bedrock Agentcore Browser tool. We will provide examples of using the browser tool headless and view the browser live.


### Tutorial Details


| Information         | Details                                                                                   |
|:--------------------|:-------------------------------------------------------------------------------------------        
| Tutorial type       | Conversational                                                                            |
| Agent type          | Single                                                                                    |
| Agentic Framework   | Broeser-Use                                                                               |
| LLM model           | Anthropic Claude 3.7 Sonnet                                                               |
| Tutorial components | Using Browser-Use SDK to interact with Bedrock Agentcore browser tool in a headless way   |
| Tutorial vertical   | Cross-vertical                                                                            |
| Example complexity  | Easy                                                                                      |
| SDK used            | Amazon BedrockAgentCore Python SDK, Browser-Use                                           |

### Tutorial Architecture

In this tutorial we will describe how to use Browser-Use SDK with Agentcore browser tool.  

In our example we will send natural language instructions to Browser-Use agent to perform tasks on the Bedrock Agentcore browser in a headless way.

<div style="text-align:left">
    <img src="images/architecture_runtime.png" width="50%"/>
</div>

### Tutorial Key Features

* Using browser tool in a headless way
* Using Browser-Use with browser tool

## Prerequisites

To execute this tutorial you will need:
* Python 3.11+
* AWS credentials
* Amazon Bedrock AgentCore SDK
* Browser-Use SDK 

## How it works

A browser tool sandbox is a secure execution environment that enables AI agents to safely interact with web browsers. When a user makes a request, the Large Language Model (LLM) selects appropriate tools and translates commands. These commands are executed within a controlled sandbox environment containing a headless browser and hosted library server (using tools like Playwright). The sandbox provides isolation and security by containing web interactions within a restricted space, preventing unauthorized system access. The agent receives feedback through screenshots and can perform automated tasks while maintaining system security. 

![architecture local](../images//browser-tool.png)

## 1. Setting Up the Environment

First, let's install and import the necessary libraries to initiaize the browser tool sandbox client. 

In [ ]:
# !pip install --force-reinstall -U -r requirements.txt --quiet

In [ ]:
# !pip install strands-agents
# !pip install strands-agents-tools
# !pip install playwright

: 

In [8]:
from strands import Agent
from strands_tools.browser import AgentCoreBrowser

agent_core_browser = AgentCoreBrowser(region="us-west-2")

print(agent_core_browser)



In [9]:
agent = Agent(
tools=[agent_core_browser.browser],
model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
)

In [10]:
def invoke(payload):
    user_message = payload.get("prompt", "")
    response = agent(user_message)
    return response.message["content"][0]["text"]



In [12]:
response = invoke(
    {
    "prompt": "Search for macbooks on apple.com and get the details of the first result"
    }
)

I'll help you search for MacBooks on Apple.com and get details about the first result. Let me do this step by step using the browser tool.

First, I'll initialize a browser session:
Tool #8: browser
Now I'll navigate to Apple.com:
Tool #9: browser
It seems there might be an issue with loading Apple.com. Let me try navigating directly to Apple's Mac section:
Tool #10: browser
Good! I've successfully loaded Apple's Mac page. Now let's look at the page content to find MacBook information:
Tool #11: browser
Let me find the MacBook links on this page:
Tool #12: browser
I see several MacBook links. The first distinct MacBook model appears to be the MacBook Air. Let's navigate to that page to get more details:
Tool #13: browser
Now let's gather details about the MacBook Air:
Tool #14: browser
Let me try to get more specific details about the MacBook Air:
Tool #15: browser
Let's try to get more specifications:
Tool #16: browser
Let me try a different approach to get pricing information:
Tool #

In [ ]:
from bedrock_agentcore.tools.browser_client import BrowserClient
from browser_use import Agent
from browser_use.browser.session import BrowserSession
from browser_use.browser import BrowserProfile
from langchain_aws import ChatBedrockConverse
from rich.console import Console
from contextlib import suppress
import asyncio

In [ ]:
console = Console()

## 2. Setup the browser client
We will setup the browser client and wait for it to be ready. Then, we will generate web-socket url and headers 
 


In [ ]:
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name

client = BrowserClient(region)
client.start()

# Extract ws_url and headers
ws_url, headers = client.generate_ws_headers()

## 4. Helper function to run browser task
Run a browser automation task using browser-use Agent 




In [ ]:
async def run_browser_task(browser_session: BrowserSession, bedrock_chat: ChatBedrockConverse, task: str) -> None:
    """
    Run a browser automation task using browser_use
    
    Args:
        browser_session: Existing browser session to reuse
        bedrock_chat: Bedrock chat model instance
        task: Natural language task for the agent
    """
    try:
        # Show task execution
        console.print(f"\n[bold blue]🤖 Executing task:[/bold blue] {task}")
        
        # Create and run the agent
        agent = Agent(
            task=task,
            llm=bedrock_chat,
            browser_session=browser_session
        )
        
        # Run with progress indicator
        with console.status("[bold green]Running browser automation...[/bold green]", spinner="dots"):
            await agent.run()
        
        console.print("[bold green]✅ Task completed successfully![/bold green]")
        
    except Exception as e:
        console.print(f"[bold red]❌ Error during task execution:[/bold red] {str(e)}")
        import traceback
        if console.is_terminal:
            traceback.print_exc()

## 5. Invoke the browser task using Browser-use profile
Create a persistent browser session using CDP WebSocket connection and initialize Bedrock Claude model for automated web tasks. Handle session lifecycle with proper cleanup and execute browser automation tasks via AI-driven commands.

In [ ]:
# Create persistent browser session and model
browser_session = None
bedrock_chat = None

try:
    # Create browser profile with headers
    browser_profile = BrowserProfile(
        headers=headers,
        timeout=1500000,  # 150 seconds timeout
    )
    
    # Create a browser session with CDP URL and keep_alive=True for persistence
    browser_session = BrowserSession(
        cdp_url=ws_url,
        browser_profile=browser_profile,
        keep_alive=True  # Keep browser alive between tasks
    )
    
    # Initialize the browser session
    console.print("[cyan]🔄 Initializing browser session...[/cyan]")
    await browser_session.start()
    
    # Create ChatBedrockConverse once
    bedrock_chat = ChatBedrockConverse(
        model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
        region_name="us-west-2"
    )
    
    console.print("[green]✅ Browser session initialized and ready for tasks[/green]\n")

    task = "Search for a coffee maker on amazon.com and extract details of the first one" ## Modify the task to run other tasks

    await run_browser_task(browser_session, bedrock_chat, task)

finally:
    # Close the browser session
    if browser_session:
        console.print("\n[yellow]🔌 Closing browser session...[/yellow]")
        with suppress(Exception):
            await browser_session.close()
        console.print("[green]✅ Browser session closed[/green]")


## 6. Cleanup
Stop the browser session if it hasn't been

In [ ]:
client.stop()
print("Browser session stopped successfully!")